<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250, style="padding: 10px"><br> 
<b>Interactive Visualization with Bokeh, HoloViews, and Datashader</b> <br>

<br>Original Author:  **Leanne Guy** ([@leannep](https://rubin-dp0/tutorial-notebooks/issues/new?body=@leannep))
<br>Last Verified: ** 2021-05-21 **
<br>Verified Stack Release: **v22.0.0**

<br> Target audience: All DP0 delegates, especially those new to Notebooks. 
<br> Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> 
<br> Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

### 2.0 Setup
You can find the Stack version by using `eups list -s` on the terminal command line.

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')

### 2.0 TAP Service

In [1]:
from rubin_jupyter_utils.lab.notebook import get_tap_service, retrieve_query    
service = get_tap_service()
assert service is not None

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


#### Simple query of the forced photometry catalog

In [5]:
maxRec = 100
query = "SELECT TOP " + str(maxRec) + " * FROM dp01_dc2_catalogs.forced_photometry"
print(query)

SELECT TOP 100 * FROM dp01_dc2_catalogs.forced_photometry


In [6]:
results = service.search(query, maxrec=maxRec)
results.to_table().show_in_notebook()

idx,coord_dec,coord_ra,g_base_ClassificationExtendedness_flag,g_base_ClassificationExtendedness_value,g_base_InputCount_flag,g_base_InputCount_flag_badCentroid,g_base_InputCount_flag_noInputs,g_base_InputCount_value,g_base_LocalBackground_flag,g_base_LocalBackground_flag_badCentroid,g_base_LocalBackground_flag_noGoodPixels,g_base_LocalBackground_flag_noPsf,g_base_LocalBackground_instFlux,g_base_LocalBackground_instFluxErr,g_base_PixelFlags_flag,g_base_PixelFlags_flag_bad,g_base_PixelFlags_flag_bright_object,g_base_PixelFlags_flag_bright_objectCenter,g_base_PixelFlags_flag_clipped,g_base_PixelFlags_flag_clippedCenter,g_base_PixelFlags_flag_cr,g_base_PixelFlags_flag_crCenter,g_base_PixelFlags_flag_edge,g_base_PixelFlags_flag_inexact_psf,g_base_PixelFlags_flag_inexact_psfCenter,g_base_PixelFlags_flag_interpolated,g_base_PixelFlags_flag_interpolatedCenter,g_base_PixelFlags_flag_offimage,g_base_PixelFlags_flag_rejected,g_base_PixelFlags_flag_rejectedCenter,g_base_PixelFlags_flag_saturated,g_base_PixelFlags_flag_saturatedCenter,g_base_PixelFlags_flag_sensor_edge,g_base_PixelFlags_flag_sensor_edgeCenter,g_base_PixelFlags_flag_suspect,g_base_PixelFlags_flag_suspectCenter,g_base_PsfFlux_apCorr,g_base_PsfFlux_apCorrErr,g_base_PsfFlux_area,g_base_PsfFlux_flag,g_base_PsfFlux_flag_apCorr,g_base_PsfFlux_flag_badCentroid,g_base_PsfFlux_flag_edge,g_base_PsfFlux_flag_noGoodPixels,g_base_PsfFlux_instFlux,g_base_PsfFlux_instFluxErr,g_base_SdssCentroid_flag,g_base_SdssCentroid_flag_almostNoSecondDerivative,g_base_SdssCentroid_flag_badInitialCentroid,g_base_SdssCentroid_flag_edge,g_base_SdssCentroid_flag_noSecondDerivative,g_base_SdssCentroid_flag_notAtMaximum,g_base_SdssCentroid_flag_resetToPeak,g_base_SdssCentroid_x,g_base_SdssCentroid_xErr,g_base_SdssCentroid_y,g_base_SdssCentroid_yErr,g_base_SdssShape_flag,g_base_SdssShape_flag_badCentroid,g_base_SdssShape_flag_maxIter,g_base_SdssShape_flag_psf,g_base_SdssShape_flag_shift,g_base_SdssShape_flag_unweighted,g_base_SdssShape_flag_unweightedBad,g_base_SdssShape_instFlux,g_base_SdssShape_instFlux_xx_Cov,g_base_SdssShape_instFlux_xy_Cov,g_base_SdssShape_instFlux_yy_Cov,g_base_SdssShape_instFluxErr,g_base_SdssShape_psf_xx,g_base_SdssShape_psf_xy,g_base_SdssShape_psf_yy,g_base_SdssShape_x,g_base_SdssShape_xx,g_base_SdssShape_xxErr,g_base_SdssShape_xy,g_base_SdssShape_xyErr,g_base_SdssShape_y,g_base_SdssShape_yy,g_base_SdssShape_yyErr,g_base_Variance_flag,g_base_Variance_flag_badCentroid,g_base_Variance_flag_emptyFootprint,g_base_Variance_value,g_good,g_modelfit_CModel_apCorr,g_modelfit_CModel_apCorrErr,g_modelfit_CModel_dev_apCorr,g_modelfit_CModel_dev_apCorrErr,g_modelfit_CModel_dev_flag,g_modelfit_CModel_dev_flag_apCorr,g_modelfit_CModel_dev_flag_badReference,g_modelfit_CModel_dev_flag_numericError,g_modelfit_CModel_dev_flux_inner,g_modelfit_CModel_dev_instFlux,g_modelfit_CModel_dev_instFluxErr,g_modelfit_CModel_exp_apCorr,g_modelfit_CModel_exp_apCorrErr,g_modelfit_CModel_exp_flag,g_modelfit_CModel_exp_flag_apCorr,g_modelfit_CModel_exp_flag_badReference,g_modelfit_CModel_exp_flag_numericError,g_modelfit_CModel_exp_flux_inner,g_modelfit_CModel_exp_instFlux,g_modelfit_CModel_exp_instFluxErr,g_modelfit_CModel_flag,g_modelfit_CModel_flag_apCorr,g_modelfit_CModel_flag_badCentroid,g_modelfit_CModel_flag_badReference,g_modelfit_CModel_flag_noShapeletPsf,g_modelfit_CModel_flag_region_maxArea,g_modelfit_CModel_flag_region_maxBadPixelFraction,g_modelfit_CModel_fracDev,g_modelfit_CModel_initial_apCorr,g_modelfit_CModel_initial_apCorrErr,g_modelfit_CModel_initial_flag,g_modelfit_CModel_initial_flag_apCorr,g_modelfit_CModel_initial_flag_badReference,g_modelfit_CModel_initial_flag_numericError,g_modelfit_CModel_initial_flux_inner,g_modelfit_CModel_initial_instFlux,g_modelfit_CModel_initial_instFluxErr,g_modelfit_CModel_instFlux,g_modelfit_CModel_instFlux_inner,g_modelfit_CModel_instFluxErr,g_modelfit_CModel_objective,i_base_ClassificationExtendedness_flag,i_base_ClassificationExtendedness_value,i_base_InputCount_f

In [ ]:
query = "SELECT count(*) FROM dp01_dc2_catalogs.forced_photometry"
results = service.search(query, maxrec=10)
results.to_table().show_in_notebook()

In [ ]:
query = "SELECT TOP 100 objectId, coord_ra, coord_dec FROM dp01_dc2_catalogs.forced_photometry"
results = service.search(query)
assert results not None
results.to_table().show_in_notebook()

In [ ]:
query = "SELECT objectId, ra, dec, mag_g, mag_r, mag_i FROM dp01_dc2_catalogs.object "  \
                          "WHERE mag_g IS NOT NULL AND mag_r IS NOT NULL and mag_i IS NOT NULL " \
                          "AND mag_g < 21.0 "
results = service.search(query, maxrec=10)
results.to_table().show_in_notebook()

In [ ]:
# catalogs 
# dp01_dc2_catalogs.object ,dp01_dc2_catalogs.position , dp01_dc2_catalogs.reference , or dp01_dc2_catalogs.truth_math

In [ ]:
query = "SELECT TOP 10 objectId, coord_ra, coord_dec FROM dp01_dc2_catalogs.position"
results = service.search(query, maxrec=10)
results.to_table().show_in_notebook()

In [ ]:
query = "SELECT TOP 10 objectId, modelfit_Cmodel_flag FROM dp01_dc2_catalogs.reference"
results = service.search(query, maxrec=10)
results.to_table().show_in_notebook()

In [ ]:
query = "SELECT TOP 10 ra, dec, flux_g, flux_r, flux_i, redshift, host_galaxy  FROM dp01_dc2_catalogs.truth_match " \
        "WHERE host_galaxy = -1"
results = service.search(query, maxrec=10)
results.to_table().show_in_notebook()

In [ ]:
query = "SELECT count(*) FROM dp01_dc2_catalogs.forced_photometry"
results = service.search(query, maxrec=10)
results.to_table().show_in_notebook()

In [ ]:
# From Dominique Boutigny
# smaller region: WHERE CONTAINS(POINT('ICRS',dpos.coord_ra,dpos.coord_dec),CIRCLE('ICRS',63.0,-38.0,10.0))=1 "\
# Large region: WHERE CONTAINS(POINT('ICRS',dpos.coord_ra,dpos.coord_dec),CIRCLE('ICRS',63.0,-38.0,30.0))=1
query = "SELECT TOP 10 dref.coord_ra as ra, dref.coord_dec as dec, "\
        " dref.ext_shapeHSM_HsmShapeRegauss_e1, dref.ext_shapeHSM_HsmShapeRegauss_e2, dref.objectId as id, dfrc.i_modelfit_CModel_instFlux " \
        "FROM dp01_dc2_catalogs.reference as dref, dp01_dc2_catalogs.position as dpos, dp01_dc2_catalogs.forced_photometry as dfrc "\
        "WHERE scisql_s2PtInCircle(dpos.coord_ra,dpos.coord_dec,63.0,-38.0,10.0)=1"\
        "AND dref.objectId = dpos.objectId AND dfrc.objectId = dpos.objectId AND dpos.detect_isPrimary = 1 " \
        "AND dfrc.i_modelfit_CModel_flag = 0 AND dfrc.i_modelfit_CModel_instFlux > 0 AND dref.base_SdssCentroid_flag = 0 "\
        "AND dref.base_Blendedness_abs < POWER(10, -0.375) AND dref.base_Blendedness_abs_instFlux IS NULL "\
        "AND dref.base_ClassificationExtendedness_flag = 0 AND dref. base_ClassificationExtendedness_value > 0 "\
        "AND ext_shapeHSM_HsmShapeRegauss_flag = 0 AND dfrc.i_modelfit_CModel_flag = 0 AND dfrc.i_modelfit_CModel_instFlux > 0 "\
        "AND dfrc.i_modelfit_CModel_instFlux/dfrc.i_modelfit_CModel_instFluxErr > 10000 "\
        "AND dref.ext_shapeHSM_HsmShapeRegauss_resolution >= 0.3 AND dref.ext_shapeHSM_HsmShapeRegauss_sigma <= 0.4 "\
        "AND SQRT(POWER(dref.ext_shapeHSM_HsmShapeRegauss_e1, 2)+POWER(dref.ext_shapeHSM_HsmShapeRegauss_e2, 2)) < 2 "\
        "AND dref.base_PixelFlags_flag_edge = 0 AND dref.base_PixelFlags_flag_interpolatedCenter = 0 "\
        "AND dref.base_PixelFlags_flag_saturatedCenter = 0 AND dref.base_PixelFlags_flag_crCenter = 0 "\
        "AND dref.base_PixelFlags_flag_bad = 0 AND dref.base_PixelFlags_flag_suspectCenter = 0 "\
        "AND dref.base_PixelFlags_flag_clipped = 0 AND dref.deblend_skipped = 0"
results = service.search(query, maxrec=10)
results.to_table().show_in_notebook()

In [ ]:
# restrict the query with 
WHERE scisql_s2PtInCircle(dpos.coord_ra,dpos.coord_dec,63.0,-38.0,10.0)=1
# or 
WHERE scisql_s2PtInCircle(dpos.coord_ra,dpos.coord_dec,63.0,-38.0,30.0)=1


In [ ]:
import numpy as np
import astropy.io.fits as pyfits

import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Range1d, HoverTool, Selection
from bokeh.plotting import figure, output_file

import holoviews as hv
from holoviews import streams
from holoviews.operation.datashader import datashade, dynspread, rasterize
from holoviews.plotting.util import process_cmap
hv.extension('bokeh')
output_notebook()

In [ ]:
infile = '/home/leannep/DATA/dr1_m2_dered_test.fits'
reader = pyfits.open(infile)
data = reader[1].data
reader.close()

data = data[data['MAG_AUTO_G_DERED'] < 26.]
data = data[1:5]
print(data)

In [ ]:
ra_target, dec_target = 323.36, -0.82

mag = data['MAG_AUTO_G_DERED']
color = data['MAG_AUTO_G_DERED'] - data['MAG_AUTO_R_DERED']

print(type(mag))

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x0=data['RA'] - ra_target,
                                    y0=data['DEC'] - dec_target,
                                    x1=color,
                                    y1=mag,
                                    ra=data['RA'],
                                    dec=data['DEC'],
                                    coadd_object_id=data['COADD_OBJECT_ID']))

In [ ]:
print(source.data)

In [ ]:
# Create a custom hover tool on both panels
hover_left = HoverTool(tooltips=[("(RA,DEC)", "(@ra, @dec)"),
                                 ("(g-r,g)", "(@x1, @y1)"),
                                 ("coadd_object_id", "@coadd_object_id")])
hover_right = HoverTool(tooltips=[("(RA,DEC)", "(@ra, @dec)"),
                                  ("(g-r,g)", "(@x1, @y1)"),
                                  ("coadd_object_id", "@coadd_object_id")])
TOOLS = "box_zoom,box_select,lasso_select,reset,help"
TOOLS_LEFT = [hover_left, TOOLS]
TOOLS_RIGHT = [hover_right, TOOLS]

#### Cone search filtered by magnitude, sorted by distance**

In [8]:
query = "SELECT  DISTANCE (POINT('ICRS', 61.863, -35.79), POINT('ICRS', ra, dec))) as dist, ra, dec, mag_g, mag_i "\
        " FROM dp01_dc2_catalogs.object " \
        " WHERE 1=CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 61.863, -35.79, 0.05555555555555555)) " \
        " AND mag_g <24 AND mag_i <24 ORDER by dist ASC"
print(query)

SELECT  DISTANCE (POINT('ICRS', 61.863, -35.79), POINT('ICRS', ra, dec))) as dist, ra, dec, mag_g, mag_i  FROM dp01_dc2_catalogs.object  WHERE 1=CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 61.863, -35.79, 0.05555555555555555))  AND mag_g <24 AND mag_i <24 ORDER by dist ASC


In [ ]:
# create a new plot and add a renderer
left = figure(tools=TOOLS_LEFT, plot_width=500, plot_height=500, output_backend="webgl",
              title='Spatial: Centered on (RA, Dec) = (%.2f, %.2f)'%(ra_target, dec_target))
left.circle('x0', 'y0', hover_color='firebrick', source=source,
            selection_fill_color='steelblue', selection_line_color='steelblue',
            nonselection_fill_color='silver', nonselection_line_color='silver')
left.x_range = Range1d(0.3, -0.3)
left.y_range = Range1d(-0.3, 0.3)
left.xaxis.axis_label = 'Delta RA'
left.yaxis.axis_label = 'Delta DEC'

# create another new plot and add a renderer
right = figure(tools=TOOLS_RIGHT, plot_width=500, plot_height=500, output_backend="webgl",
               title='CMD')
right.circle('x1', 'y1', hover_color='firebrick', source=source,
             selection_fill_color='steelblue', selection_line_color='steelblue',
             nonselection_fill_color='silver', nonselection_line_color='silver')
right.x_range = Range1d(-0.5, 2.5)
right.y_range = Range1d(26., 16.)
right.xaxis.axis_label = 'g - r'
right.yaxis.axis_label = 'g'

p = gridplot([[left, right]])

# The plots can be exported as html files with data embedded
#output_file("bokeh_m2_example.html", title="M2 Example")

show(p)

In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts
from holoviews import streams
hv.extension('bokeh')

In [ ]:
# Declare distribution of Points
points = hv.Points(np.random.multivariate_normal((0, 0), [[1, 0.1], [0.1, 1]], (1000,)))

# Declare points selection selection
sel = streams.Selection1D(source=points)

# Declare DynamicMap computing mean y-value of selection
mean_sel = hv.DynamicMap(lambda index: hv.HLine(points['y'][index].mean() if index else -10),
                         kdims=[], streams=[sel])

# Declare a Bounds stream and DynamicMap to get box_select geometry and draw it
box = streams.BoundsXY(source=points, bounds=(0,0,0,0))
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Declare DynamicMap to apply bounds selection
dmap = hv.DynamicMap(lambda bounds: points.select(x=(bounds[0], bounds[2]),
                                                  y=(bounds[1], bounds[3])),
                     streams=[box])

# Compute histograms of selection along x-axis and y-axis
yhist = hv.operation.histogram(dmap, bin_range=points.range('y'), dimension='y', dynamic=True, normed=False)
xhist = hv.operation.histogram(dmap, bin_range=points.range('x'), dimension='x', dynamic=True, normed=False)

# Combine components and display
points * mean_sel * bounds << yhist << xhist

#### Query the forced source table to build a light curve

In [16]:
query = "SELECT * FROM tap_schema.tables WHERE tap_schema.tables.table_name like 'desc%' "
print(query)
results = service.search(query, maxrec = 100)
results.to_table().show_in_notebook()

SELECT * FROM tap_schema.tables WHERE tap_schema.tables.table_name like 'desc%' 


idx,description,schema_name,table_index,table_name,table_type,utype
0,,desc_dc2_run12p_v4e,0,desc_dc2_run12p_v4e.ccdvisit,table,
1,Forced-photometry source measurement on an individual Exposure based on a Multifit shape model derived from a deep detection.,desc_dc2_run12p_v4e,0,desc_dc2_run12p_v4e.forcedsource,table,
2,"The Object table contains descriptions of the multi-epoch static astronomical objects, in particular their astrophysical properties as derived from analysis of the Sources that are associated with them.",desc_dc2_run12p_v4e,0,desc_dc2_run12p_v4e.object,table,


In [19]:
forcedSourceTable ='desc_dc2_run12p_v4e'
query = "SELECT TOP 10 * FROM " + forcedSourceTable
results = service.search(query, maxrec = 100)
results.to_table().show_in_notebook()

DALQueryError: Table [ desc_dc2_run12p_v4e ] is not found in TapSchema. Possible reasons: table does not exist or permission is denied.

In [ ]:
SELECT obj.ra as ora, obj.dec as odec,
    truth.ra as tra, truth.dec as tdec, truth.truth_type, truth.redshift, truth.is_variable,
    obj.mag_g as g, obj.mag_i as i, obj.mag_r as r, truth.mag_r as tmr
    FROM dp01_dc2_catalogs.object as obj
    JOIN dp01_dc2_catalogs.truth_match as truth
    ON truth.match_objectId = obj.objectId
    WHERE CONTAINS(
    POINT('ICRS', obj.ra, obj.dec),
    CIRCLE('ICRS', 61.863, -35.79, 0.05555555555555555))=1
    AND (obj.mag_g <24 AND obj.mag_i <24)
    AND truth.is_good_match = 1